In [ ]:
import pandas as pd
from IPython.display import display 

data = pd.read_csv('../data/vehicle_price_prediction.csv',sep=',')
# Exemplo: remover as últimas 500 linhas
data = data.iloc[:-100000]

# Salvar em novo arquivo
data.to_csv("vehicle_price_prediction.csv", index=False)

